<a href="https://colab.research.google.com/github/Shruti12377/Numpy_notes/blob/main/PRODIGY_ML_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Download and extract dataset
!kaggle datasets download -d gti-upm/leapgestrecog
with zipfile.ZipFile('leapgestrecog.zip', 'r') as zip_ref:
    zip_ref.extractall('leapgestrecog')

# Directory structure
data_dir = 'leapgestrecog/leapGestRecog'

# Define image size and batch size
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

# Load and preprocess the dataset
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
 99% 2.11G/2.13G [00:23<00:00, 60.7MB/s]
100% 2.13G/2.13G [00:23<00:00, 95.4MB/s]
Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20
)


Epoch 1/20
500/500 [==============================] - 616s 1s/step - loss: 0.0249 - accuracy: 0.9877 - val_loss: 0.9576 - val_accuracy: 0.8440
Epoch 2/20
500/500 [==============================] - 652s 1s/step - loss: 0.0270 - accuracy: 0.9865 - val_loss: 0.9748 - val_accuracy: 0.8320
Epoch 3/20
 23/500 [>.............................] - ETA: 8:47 - loss: 0.0134 - accuracy: 0.9932

In [ ]:
# Evaluate the model
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc:.2f}')


In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('gesture_model.tflite', 'wb') as f:
    f.write(tflite_model)
